## Chapter 5: Working with Forms and User Input

Most web applications are not just about displaying data—they need to accept input from users. Whether it's a login form, a product creation page, or a search box, handling user input securely and efficiently is a core skill for any ASP.NET developer. In this chapter, you'll learn how to create HTML forms in Razor, how ASP.NET Core automatically maps form data to C# objects (model binding), how to handle form submissions in controllers, and how to use data annotations to provide hints for UI generation. We'll also touch on validation basics, preparing you for the deeper dive in Chapter 6.

### 5.1 Creating HTML Forms in Razor

Forms are the primary way users send data to the server. In ASP.NET Core MVC, you can create forms using standard HTML or, better yet, using Tag Helpers that integrate with your models and routing.

#### The `<form>` Tag Helper

The Form Tag Helper simplifies form creation by automatically generating the correct `action` attribute and adding an anti-forgery token.

**Basic example:**

```html
<form asp-action="Create" asp-controller="Products" method="post">
    <!-- form fields go here -->
    <button type="submit">Save</button>
</form>
```

This generates HTML similar to:

```html
<form method="post" action="/Products/Create">
    <input name="__RequestVerificationToken" type="hidden" value="...">
    <!-- form fields -->
    <button type="submit">Save</button>
</form>
```

The anti-forgery token helps prevent Cross-Site Request Forgery (CSRF) attacks. It's automatically validated on the server when you decorate your action with `[ValidateAntiForgeryToken]` (more on that later).

#### Common Form Fields with Tag Helpers

Tag Helpers exist for most HTML input types, making it easy to bind them to model properties.

**Input Tag Helper** (`<input asp-for="PropertyName">`):
- Renders an `<input>` element with appropriate `type` based on the property's data type (e.g., `type="text"` for string, `type="number"` for numeric types, `type="checkbox"` for bool).
- Sets the `id` and `name` attributes to the property name.
- Adds `data-val-*` attributes for client-side validation if enabled.

**Example:**

```html
<div class="form-group">
    <label asp-for="Name"></label>
    <input asp-for="Name" class="form-control" />
</div>
```

**Label Tag Helper** (`<label asp-for="PropertyName"></label>`):
- Renders a `<label>` element with the `for` attribute set to the input's ID.
- By default, displays the property name; you can customize with `[Display]` attribute.

**Textarea Tag Helper** (`<textarea asp-for="PropertyName"></textarea>`):
- Renders a `<textarea>` for multi-line text.

**Select Tag Helper** (`<select asp-for="PropertyName" asp-items="SelectList"></select>`):
- Renders a dropdown list. The `asp-items` attribute accepts an `IEnumerable<SelectListItem>`.

**Example with dropdown:**

```csharp
// In controller action that displays the form
ViewBag.CategoryList = new SelectList(categories, "Id", "Name");
```

```html
<select asp-for="CategoryId" asp-items="ViewBag.CategoryList" class="form-control">
    <option value="">-- Select Category --</option>
</select>
```

**Validation Message Tag Helper** (`<span asp-validation-for="PropertyName"></span>`):
- Displays validation errors for a specific property. We'll use it extensively in Chapter 6.

#### A Complete Form Example

Let's create a form for adding a new product, using the `Product` model from previous chapters.

**`ProductsController.cs` – GET action:**

```csharp
public IActionResult Create()
{
    // Populate categories for dropdown (simulated)
    var categories = new List<Category>
    {
        new Category { Id = 1, Name = "Electronics" },
        new Category { Id = 2, Name = "Books" },
        new Category { Id = 3, Name = "Clothing" }
    };
    ViewBag.CategoryList = new SelectList(categories, "Id", "Name");
    return View();
}
```

**`Views/Products/Create.cshtml`:**

```html
@model Product

<h1>Create Product</h1>

<form asp-action="Create" method="post">
    <div class="form-group">
        <label asp-for="Name"></label>
        <input asp-for="Name" class="form-control" />
        <span asp-validation-for="Name" class="text-danger"></span>
    </div>

    <div class="form-group">
        <label asp-for="Price"></label>
        <input asp-for="Price" class="form-control" />
        <span asp-validation-for="Price" class="text-danger"></span>
    </div>

    <div class="form-group">
        <label asp-for="Description"></label>
        <textarea asp-for="Description" class="form-control" rows="3"></textarea>
    </div>

    <div class="form-group">
        <label asp-for="CategoryId"></label>
        <select asp-for="CategoryId" asp-items="ViewBag.CategoryList" class="form-control">
            <option value="">-- Select Category --</option>
        </select>
        <span asp-validation-for="CategoryId" class="text-danger"></span>
    </div>

    <div class="form-group form-check">
        <input asp-for="IsAvailable" class="form-check-input" />
        <label asp-for="IsAvailable" class="form-check-label"></label>
    </div>

    <button type="submit" class="btn btn-primary">Create</button>
    <a asp-action="Index" class="btn btn-secondary">Cancel</a>
</form>
```

**Note:** We haven't added `IsAvailable` to our `Product` model yet. Let's update it:

```csharp
public class Product
{
    public int Id { get; set; }
    public string Name { get; set; }
    public decimal Price { get; set; }
    public string Description { get; set; }
    public int CategoryId { get; set; }
    public bool IsAvailable { get; set; }
}
```

This form, when submitted, will POST the data to the `Create` action.

---

### 5.2 Model Binding: How ASP.NET Maps Form Data to C# Objects

**Model binding** is the process by which ASP.NET Core takes incoming HTTP data (from form fields, query strings, route data, etc.) and converts it into .NET objects that your action methods can use. It's a powerful feature that eliminates tedious manual parsing.

#### How Model Binding Works

When an action method executes, the model binding system looks at the parameters the method expects. It then searches for values in the following sources (in order):

1. **Form data** (from `application/x-www-form-urlencoded` or `multipart/form-data` POST requests)
2. **Route values** (from the route template, e.g., `{id}`)
3. **Query string** (e.g., `?page=2`)
4. **Uploaded files** (for `IFormFile` parameters)

If a parameter is a simple type (int, string, bool, etc.), model binding tries to match the parameter name to a value from these sources. If the parameter is a complex type (like `Product`), it recursively binds properties of that type.

#### Binding to Simple Parameters

Consider this action:

```csharp
public IActionResult Search(string category, int page = 1)
{
    // ...
}
```

A request to `/products/search?category=electronics&page=2` will bind `category` to "electronics" and `page` to 2. If `page` is omitted, it defaults to 1.

#### Binding to Complex Types

For our `Create` POST action, we'll receive a `Product` object:

```csharp
[HttpPost]
public IActionResult Create(Product product)
{
    // product is automatically populated from form fields
    // ...
}
```

Model binding will:
- Look for form fields named `Name`, `Price`, `Description`, `CategoryId`, `IsAvailable`.
- Convert the values to the appropriate types (string to decimal, "on" to bool for checkbox, etc.).
- Create a new `Product` instance and set its properties.

#### Customizing Binding with Attributes

You can fine-tune model binding using attributes:

- `[BindProperty]` – applied to controller properties to indicate they should be model-bound (useful for properties that persist across requests).
- `[FromRoute]`, `[FromQuery]`, `[FromForm]`, `[FromBody]` – specify the exact source for a parameter.
- `[Bind]` – include/exclude specific properties from binding (use with caution for security).

**Example:**

```csharp
public IActionResult Update([FromRoute] int id, [FromBody] Product product)
{
    // id comes from route, product from JSON body
}
```

#### Model Binding and Security

By default, model binding binds all public properties of a complex type. This can be a security risk if your model includes properties that should not be set by the user (e.g., `IsAdmin`). To prevent over-posting, use the `[Bind]` attribute or employ view models that only contain properties you expect to be set.

**Example of safe practice with a view model:**

```csharp
public class ProductCreateViewModel
{
    public string Name { get; set; }
    public decimal Price { get; set; }
    public string Description { get; set; }
    public int CategoryId { get; set; }
    public bool IsAvailable { get; set; }
}

[HttpPost]
public IActionResult Create(ProductCreateViewModel viewModel)
{
    // map viewModel to actual Product entity
}
```

#### Binding to Collections and Dictionaries

Model binding also supports arrays, lists, and dictionaries. For example, to bind a list of IDs:

```html
<input type="checkbox" name="selectedIds" value="1" />
<input type="checkbox" name="selectedIds" value="2" />
```

Action:

```csharp
public IActionResult DeleteSelected(int[] selectedIds)
```

For complex collections, the name must follow a convention: `items[0].Name`, `items[1].Name`. ASP.NET Core can bind to `List<Product>` if the form fields are named accordingly.

---

### 5.3 Handling Form Submissions (POST)

Now that you can create forms and understand model binding, let's focus on the controller side: handling the POST request, validating the data, and responding appropriately.

#### The POST Action

In the `ProductsController`, add the following action:

```csharp
[HttpPost]
[ValidateAntiForgeryToken]
public IActionResult Create(Product product)
{
    if (ModelState.IsValid)
    {
        // Save to database (simulated)
        product.Id = _products.Max(p => p.Id) + 1;
        _products.Add(product);
        TempData["SuccessMessage"] = "Product created successfully!";
        return RedirectToAction(nameof(Index));
    }

    // If we got here, something went wrong; redisplay form with errors
    // Repopulate categories for dropdown
    var categories = new List<Category> { /* ... */ };
    ViewBag.CategoryList = new SelectList(categories, "Id", "Name");
    return View(product);
}
```

Let's dissect this:

- `[HttpPost]` – ensures this action only handles POST requests.
- `[ValidateAntiForgeryToken]` – validates the anti-forgery token that was automatically added by the Form Tag Helper. This is crucial for security.
- `ModelState.IsValid` – checks if the model passed validation rules. We haven't added validation attributes yet, so it will always be true for now. We'll fix that shortly.
- If valid: save the product, set a success message in `TempData`, and redirect to the Index page. This follows the **Post-Redirect-Get (PRG)** pattern, which prevents duplicate form submissions if the user refreshes the page.
- If invalid: redisplay the form with the user's entered data (so they can correct mistakes) and show validation errors. We must repopulate any dropdown data (`ViewBag.CategoryList`) because it's not preserved across requests.

#### The Post-Redirect-Get (PRG) Pattern

PRG is a best practice for handling form submissions:

1. **POST** – The user submits the form.
2. **Redirect** – The server processes the data and issues a redirect (HTTP 302) to a GET endpoint.
3. **GET** – The browser requests the page, which now reflects the new state.

Benefits:
- Prevents duplicate submissions on page refresh.
- Keeps the URL clean and bookmarkable.
- Improves user experience.

#### ModelState

`ModelState` is a dictionary that tracks validation errors and the submitted values. After model binding, it contains:
- The raw submitted values.
- Any validation errors added by data annotations or custom logic.
- A flag indicating whether the model is valid.

You can add custom errors to `ModelState`:

```csharp
if (product.Price < 0)
{
    ModelState.AddModelError(nameof(product.Price), "Price cannot be negative.");
}
```

In the view, `asp-validation-for` will display these errors.

#### Handling File Uploads

If your form includes file uploads, you need to use `multipart/form-data` encoding and bind to `IFormFile` properties.

**Model:**

```csharp
public class ProductWithImage
{
    public string Name { get; set; }
    public IFormFile Image { get; set; }
}
```

**Form:**

```html
<form asp-action="Create" method="post" enctype="multipart/form-data">
    <input asp-for="Name" />
    <input asp-for="Image" type="file" />
    <button type="submit">Upload</button>
</form>
```

**Action:**

```csharp
[HttpPost]
public async Task<IActionResult> Create(ProductWithImage model)
{
    if (model.Image != null && model.Image.Length > 0)
    {
        var filePath = Path.Combine("wwwroot/images", model.Image.FileName);
        using (var stream = new FileStream(filePath, FileMode.Create))
        {
            await model.Image.CopyToAsync(stream);
        }
    }
    // ...
}
```

**Important:** Always validate file size and type to prevent denial-of-service attacks.

---

### 5.4 Introduction to Data Annotations for UI Hints

Data annotations are attributes you apply to model properties to control validation, display formatting, and how the UI is generated. While Chapter 6 will cover validation in depth, we'll introduce the most common annotations here, especially those that affect the UI.

#### Display Attributes

- `[Display(Name = "Product Name")]` – sets the label text in `<label asp-for>`.
- `[DisplayFormat(DataFormatString = "{0:C}")]` – specifies how to format the value (e.g., currency).
- `[DataType(DataType.Date)]` – hints at the data type, which can affect the HTML5 input type (e.g., `type="date"`).

**Example:**

```csharp
public class Product
{
    public int Id { get; set; }

    [Display(Name = "Product Name")]
    public string Name { get; set; }

    [DisplayFormat(DataFormatString = "{0:C}")]
    public decimal Price { get; set; }

    [DataType(DataType.MultilineText)]
    public string Description { get; set; }
}
```

#### Validation Attributes (Preview)

- `[Required]` – ensures the field is not null or empty.
- `[StringLength(100)]` – limits string length.
- `[Range(0.01, 9999.99)]` – restricts numeric values.
- `[EmailAddress]` – validates email format.
- `[Compare("OtherProperty")]` – ensures two properties match (e.g., password confirmation).
- `[RegularExpression(@"pattern")]` – matches a regex.

**Example with validation:**

```csharp
public class Product
{
    public int Id { get; set; }

    [Required(ErrorMessage = "Product name is required.")]
    [StringLength(100, MinimumLength = 3)]
    [Display(Name = "Product Name")]
    public string Name { get; set; }

    [Required]
    [Range(0.01, 10000.00, ErrorMessage = "Price must be between 0.01 and 10000.00")]
    [DisplayFormat(DataFormatString = "{0:C}")]
    public decimal Price { get; set; }

    [DataType(DataType.MultilineText)]
    public string Description { get; set; }

    [Required]
    [Display(Name = "Category")]
    public int CategoryId { get; set; }

    [Display(Name = "Available?")]
    public bool IsAvailable { get; set; }
}
```

When you use Tag Helpers like `<input asp-for="Name">`, these attributes:
- Generate HTML5 validation attributes (`data-val-required`, `data-val-length`, etc.) for client-side validation (if enabled).
- Are used by server-side validation (via `ModelState.IsValid`).

#### Enabling Client-Side Validation

Client-side validation provides immediate feedback without a round trip to the server. It's enabled by default in new projects because the `_ValidationScriptsPartial.cshtml` is included in the layout or view.

Typically, you include validation scripts in your layout or view:

```html
@section Scripts {
    <partial name="_ValidationScriptsPartial" />
}
```

This partial includes jQuery Validation and jQuery Unobtrusive Validation, which automatically parse the `data-val-*` attributes generated by Tag Helpers.

#### Customizing Error Messages

You can set custom error messages in the attributes, as shown above. You can also use resource files for localization.

---

### 5.5 A Complete Form Handling Example with Validation

Let's enhance our product creation flow with data annotations and validation.

**Updated `Product` model with annotations:**

```csharp
using System.ComponentModel.DataAnnotations;

public class Product
{
    public int Id { get; set; }

    [Required(ErrorMessage = "Product name is required.")]
    [StringLength(100, MinimumLength = 3, ErrorMessage = "Name must be between 3 and 100 characters.")]
    [Display(Name = "Product Name")]
    public string Name { get; set; }

    [Required(ErrorMessage = "Price is required.")]
    [Range(0.01, 10000.00, ErrorMessage = "Price must be between 0.01 and 10000.00.")]
    [DisplayFormat(DataFormatString = "{0:C}")]
    public decimal Price { get; set; }

    [DataType(DataType.MultilineText)]
    public string Description { get; set; }

    [Required(ErrorMessage = "Please select a category.")]
    [Display(Name = "Category")]
    public int CategoryId { get; set; }

    [Display(Name = "Available for purchase?")]
    public bool IsAvailable { get; set; }
}
```

**Updated `Create` POST action with validation:**

```csharp
[HttpPost]
[ValidateAntiForgeryToken]
public IActionResult Create(Product product)
{
    if (ModelState.IsValid)
    {
        // Simulate save
        product.Id = _products.Max(p => p.Id) + 1;
        _products.Add(product);
        TempData["SuccessMessage"] = "Product created successfully!";
        return RedirectToAction(nameof(Index));
    }

    // Repopulate categories dropdown
    var categories = new List<Category>
    {
        new Category { Id = 1, Name = "Electronics" },
        new Category { Id = 2, Name = "Books" },
        new Category { Id = 3, Name = "Clothing" }
    };
    ViewBag.CategoryList = new SelectList(categories, "Id", "Name");
    return View(product);
}
```

**View `Create.cshtml` with validation placeholders:**

We already included `<span asp-validation-for>` in our form. Ensure that the validation scripts are included, either in the layout or by adding the following at the bottom of the view:

```html
@section Scripts {
    <partial name="_ValidationScriptsPartial" />
}
```

Now, when the user submits an invalid form, the page will redisplay with validation errors, both client-side (if JavaScript is enabled) and server-side.

**Testing the flow:**

1. Navigate to `/products/create`.
2. Submit the form empty → you'll see validation messages.
3. Fill it correctly → after submit, you're redirected to the product list with a success message.

---

### Summary

You've learned the essentials of working with forms and user input in ASP.NET Core MVC:

- Creating forms with Tag Helpers (`<form asp-action>`, `<input asp-for>`, etc.).
- Understanding model binding and how data flows from HTTP to C# objects.
- Handling POST submissions with the PRG pattern and anti-forgery protection.
- Using data annotations to provide UI hints and basic validation.

This foundation is critical for any interactive web application. In the next chapter, **"Model Validation,"** we'll dive deep into validation: server-side validation, custom validation attributes, remote validation, and best practices for ensuring data integrity and security. You'll learn how to build robust forms that protect your application from invalid or malicious input.

**Exercise:**
1. Extend the product form to include a file upload for a product image (use `IFormFile`).
2. Add validation to ensure the uploaded file is an image (check content type) and does not exceed 2MB.
3. Display the uploaded image on the product list page (you may need to store the file path in the model).
4. Add a new property `ReleaseDate` of type `DateTime` to the `Product` model, and use `[DataType(DataType.Date)]` to render a date picker. Test validation with future dates (you may need a custom validation attribute, which we'll cover next chapter).